In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from functools import reduce

In [ ]:
import sys

In [ ]:
sys.path.append('/Users/jomammahoule1993/Downloads/gps_wind_station-master') # add path where the following python modules live

In [ ]:
import load_windgps_data_to_pandas
import process_windgps_data

In [ ]:
# where your binary data files are
data_directoryA = '/Users/jomammahoule1993/Downloads/20201020_blackrock/A/'
data_directoryB = '/Users/jomammahoule1993/Downloads/20201020_blackrock/B/'
#data_directoryC = '/Users/jomammahoule1993/Downloads/20201020_blackrock/C/' #no data - recording malfunction
data_directoryD = '/Users/jomammahoule1993/Downloads/20201020_blackrock/D/'
data_directoryE = '/Users/jomammahoule1993/Downloads/20201020_blackrock/E/'
data_directoryF = '/Users/jomammahoule1993/Downloads/20201020_blackrock/F/'
data_directoryG = '/Users/jomammahoule1993/Downloads/20201020_blackrock/G/'
data_directoryH = '/Users/jomammahoule1993/Downloads/20201020_blackrock/H/'
data_directoryI = '/Users/jomammahoule1993/Downloads/20201020_blackrock/I/'

In [ ]:
correct_year = 2020
out = '.' # in this directory

In [ ]:
df1 = load_windgps_data_to_pandas.load_data_from_directory(data_directoryA)
df2 = load_windgps_data_to_pandas.load_data_from_directory(data_directoryB)
#df3 = load_windgps_data_to_pandas.load_data_from_directory(data_directoryC)
df4 = load_windgps_data_to_pandas.load_data_from_directory(data_directoryD)
df5 = load_windgps_data_to_pandas.load_data_from_directory(data_directoryE)
df6 = load_windgps_data_to_pandas.load_data_from_directory(data_directoryF)
df7 = load_windgps_data_to_pandas.load_data_from_directory(data_directoryG)
df8 = load_windgps_data_to_pandas.load_data_from_directory(data_directoryH)
df9 = load_windgps_data_to_pandas.load_data_from_directory(data_directoryI)

In [ ]:
alldfs=[df1,df2,df4,df5,df6,df7,df8,df9]

In [ ]:
for i in range(0,len(alldfs)):
    alldfs[i] = process_windgps_data.fix_millis_errors(alldfs[i])
    alldfs[i] = process_windgps_data.fix_gps_date(alldfs[i], correct_year=correct_year)
    alldfs[i] = process_windgps_data.calc_interpolated_epoch_time(alldfs[i])
    plt.plot(alldfs[i].millis, alldfs[i].time_epoch)

In [ ]:
for i in range(0,len(alldfs)):
    alldfs[i] = process_windgps_data.parse_and_save_several_wind_strings(alldfs[i], wind_strings=['S2', 'D', 'T', 'U', 'V', 'W'])


In [ ]:
#save files as hdfs
names=['SensorA','SensorB','SensorD', 'SensorE', 'SensorF', 'SensorG', 'SensorH', 'SensorI']
for i in range(0,len(alldfs)):
    alldfs[i].to_hdf(names[i] + '_10.20.20', key='/windgps')


In [ ]:
#read in hdfs
df1=pd.read_hdf('SensorA_10.20.20')
df2=pd.read_hdf('SensorB_10.20.20')
df4=pd.read_hdf('SensorD_10.20.20')
df5=pd.read_hdf('SensorE_10.20.20')
df6=pd.read_hdf('SensorF_10.20.20')
df7=pd.read_hdf('SensorG_10.20.20')
df8=pd.read_hdf('SensorH_10.20.20')
df9=pd.read_hdf('SensorI_10.20.20')
df1

In [ ]:
#ADD suffixes to each DF for all values except time_epoch

df1=df1.add_suffix('_1')
df1=df1.rename(columns={"time_epoch_1":"time"})

df2=df2.add_suffix('_2')
df2=df2.rename(columns={"time_epoch_2":"time"})

#df3=df3.add_suffix('_3')
#df3=df3.rename(columns={"time_epoch_3":"time"})

df4=df4.add_suffix('_4')
df4=df4.rename(columns={"time_epoch_4":"time"})

df5=df5.add_suffix('_5')
df5=df5.rename(columns={"time_epoch_5":"time"})

df6=df6.add_suffix('_6')
df6=df6.rename(columns={"time_epoch_6":"time"})

df7=df7.add_suffix('_7')
df7=df7.rename(columns={"time_epoch_7":"time"})

df8=df8.add_suffix('_8')
df8=df8.rename(columns={"time_epoch_8":"time"})

df9=df9.add_suffix('_9')
df9=df9.rename(columns={"time_epoch_9":"time"})

In [ ]:
dfs=[df1,df2,df4,df5,df6,df7,df8,df9]

In [ ]:
#round all dfs to 1 decimal (their sampling rate)
decimals = 1
for i in range(0,len(alldfs)):
    dfs[i]['time'] = dfs[i]['time'].apply(lambda x: round(x, decimals))


In [ ]:
df_merged = reduce(lambda  left, right: pd.merge(left, right,on=['time'],
                                            how='inner'), dfs)

In [ ]:
df_merged = df_merged.interpolate()

In [ ]:
df_merged

In [ ]:
plt.plot(df_merged.time)

In [ ]:
dt_wind = np.median(np.diff(df_merged.time.dropna().values))
print('dt Wind: ', dt_wind)

first_time = np.min([df_merged.time])
last_time = np.max([df_merged.time])
print(first_time,last_time)
master_time = np.arange(first_time, last_time, dt_wind)


In [ ]:
fnameMerged = 'Merged_10.20.20' + 'windgps_data.hdf'
full_fnameMerged = os.path.join('/Users/Desktop/', fnameMerged)
df_merged.to_hdf(full_fnameMerged, 'windgps')

In [ ]:
dfsmall=df_merged[["time","D_2","D_5","D_6", "D_7", "D_8", "D_9", "S2_2", "S2_5", "S2_6", "S2_7", "S2_8", "S2_9"]]

In [ ]:
fnameSmall = 'SpeedandDirection_10.20.20' + 'windgps_data.hdf'
full_fnameSmall = os.path.join('/Users/Desktop/', fnameSmall)
dfsmall.to_hdf(full_fnameSmall, 'windgps')

In [ ]:
dflatlon=df_merged[["time", "lat_2", "lon_2", "lat_5", "lon_5", "lat_6", "lon_6",  "lat_7", "lon_7", "lat_8", "lon_8", "lat_9", "lon_9"]]

In [ ]:
fnameLatLons = 'LatLons_10.20.20' + 'windgps_data.hdf'
full_fnameLatLons = os.path.join('/Users/Desktop/', fnameLatLons)
dflatlon.to_hdf(full_fnameLatLons, 'windgps')
